# Facebook research의 faiss를 이용한 similarity search
- Vector similarity search
- 유사 이미지 검색
- Feature extrator를 이용해 CNN기법 활용

# faiss install colab - Colab에서 설치하기 위한 준비 수행
- https://stackoverflow.com/questions/47967252/installing-faiss-on-google-colaboratory

In [ ]:
!apt install libomp-dev
!python -m pip install --upgrade faiss # faiss-gpu

In [ ]:
import numpy as np
from PIL import Image
import csv
import os
from os import listdir
from os.path import isfile, join, splitext
import shutil
import random
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import faiss
import time

%matplotlib inline

In [ ]:
# 기초 예제
d = 1
nb = 5                      # 데이터셋 사이크
nq = 1                     # 쿼리
np.random.seed(1234)             # 랜덤 적용

xb = np.random.random_integers(10, size=(nb,d)).astype('float32')  # 반드시 float 형태이어야 함
xq = np.random.random_integers(10, size=(nq,d)).astype('float32')


In [ ]:
print('데이터셋 xb: ', xb)
print('쿼리 xq: ', xq)

In [ ]:
index = faiss.IndexFlatL2(d)   # 인덱스 생성
print('index.is_trained : ', index.is_trained)

index.add(xb)                  # 인덱스에 데이터셋 추가
print('index.ntotal : ', index.ntotal)

In [ ]:
k = 3  # 최근접 3개의 결과 출력
D, I = index.search(xq, k) 
print('I : ', I)  # index 번호
print('D : ', D)  # distance

In [ ]:
# Index에 따른 데이터 출력
print('데이터: ', xb)
print('쿼리: ', xq)
for _, idx in enumerate(I[0]):
    print('최근접 데이터 : ', xb[idx])

# 이미지를 vector(array)로 변환
- 이미지 로드
- numpy array로 변환
- array를 다시 image로 변환

In [ ]:
from PIL import Image
import numpy as np
import urllib.request

urllib.request.urlretrieve(
  '<CHANGE_HERE>',
   "apple.jpg")
  
img = Image.open("apple.jpg")
display(img)


In [ ]:
#resize image
size=(200,200)
resized_img = img.resize(size)
display(resized_img)

In [ ]:
im2arr = np.array(resized_img) # im2arr.shape: height x width x channel
print('이미지 배열: ', im2arr[0:2])

In [ ]:
print('배열 구조: ', im2arr.shape)
flat_im2arr = im2arr.flatten()  # 1차원 배열로 변환 - https://numpy.org/doc/stable/reference/generated/numpy.ndarray.flatten.html
print('배열 flatten: ', flat_im2arr)

In [ ]:
im2arr.shape

In [ ]:
arr2im = Image.fromarray(im2arr)  # array를 다시 이미지로 변환
display(arr2im)

# Caltec101 이미지 파일 다운로드
- [Caltec101 데이터셋](http://www.vision.caltech.edu/Image_Datasets/Caltech101/)
- 101 카테고리(class)
- 카테고리별 40~800 이미지 파일
- 이미지 크기는 300*200 픽셀(가변적)

In [ ]:
!wget -O "101_ObjectCategories.tar.gz" "<CHANGE_HERE>"

In [ ]:
!ls -al

In [ ]:
# Unzip the data
!tar xvzf 101_ObjectCategories.tar.gz > /dev/null  # silence - tar 결과 출력 제거
# !rm 101_ObjectCategories.tar.gz*

In [ ]:
import glob
import numpy as np
from PIL import Image


def get_filenames(glob_pattern, recursive=True):
    """Extracts list of filenames (full paths) based on specific glob path pattern.
    
    Parameters
    ----------
    glob_pattern : str
        Glob pattern for glob to extract filenames, eg. "directory/**/*.jpg"
    recursive : bool, optional
        Recursively search through subdirectories, by default True
    
    Returns
    -------
    list
        List of file paths
    """
    all_files = glob.glob(glob_pattern, recursive=recursive)
    print('Found %s files using pattern: %s' % (len(all_files), glob_pattern))
    return all_files


def expand2square(pil_img, background_color):
    """Function to pad an image to square using specific bg clr.
    
    Parameters
    ----------
    pil_img : PIL.Image
        Pillow Image object that should be processed
    background_color : int
        Integer value representing bg color
    
    Returns
    -------
    PIL.Image
        Square-padded image object
    """
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result


def get_images(filenames, target_size=(200,200), color='RGB', bg_clr=0):
    """Reads image files from provided file paths list, applies square-padding,
    resizes all images into target size and returns them as a single numpy array
    
    Parameters
    ----------
    filenames : list
        List of image file paths
    target_size : tuple, optional
        Target size for all the images to be resized to, by default (200,200)
    color : str, optional
        Color mode strategy for PIL when loading images, by default 'RGB'
    bg_clr : int, optional
        Integer representing background color used for square-padding, by default 0
    
    Returns
    -------
    numpy.array
        Numpy array with resized images
    """
    imgs_list = []
    for filename in filenames:
        img = Image.open(filename).convert(color)
        im_square = expand2square(img, bg_clr)
        im_res = im_square.resize(target_size)
        imgs_list.append(np.array(im_res))

    return np.asarray(imgs_list)


In [ ]:
# Caltech101 dataset 파일 path list 생성
filenames = get_filenames("101_ObjectCategories//**//*.*")
filenames[0:10]  # 전체 파일 리스트 중 10개만 조회

In [ ]:
# numpy array로 이미지 파일을 변환
imgs_np = get_images(filenames, target_size=(200,200), color='RGB', bg_clr=0)

In [ ]:
imgs_np.shape

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(imgs_np[900])  # 900번째 array 이미지 출력

In [ ]:
# GPU가 아닐 경우 array 데이터셋 수를 조절. 1000개만 사용 
imgs_np = imgs_np[:1000]
# imgs_np = imgs_np  # 전체 데이터셋
imgs_np.shape

# Extract feature
- 이미지의 feature를 pretrained model로 추출
- feature를 이미지별로 2048 size 출력
- VGG19와 Inception_V3 pretrained model 사용

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.inception_v3 import InceptionV3

def create_feat_extractor(base_model, pooling_method='avg'):
    """Creates a features extractor based on the provided base network.
    
    Parameters
    ----------
    base_model : keras.Model
        Base network for feature extraction
    pooling_method : str, optional
        Pooling method that will be used as the last layer, by default 'avg'
    
    Returns
    -------
    keras.Model
        Ready to use feature extractor
    """
    assert pooling_method in ['avg', 'max']
    
    x = base_model.output
    if pooling_method=='avg':
        x = GlobalAveragePooling2D()(x)
    elif pooling_method=='max':
        x = GlobalMaxPooling2D()(x)
    # model = Model(input=base_model.input, output=[x])
    model = Model(base_model.input, [x])  # https://github.com/keras-team/keras/issues/13743#issuecomment-609674110

    return model


def extract_features(imgs_np, pretrained_model="resnet50", pooling_method='avg'):    
    """Takes in an array of fixed size images and returns features/embeddings
    returned by one of the selected pretrained networks.
    
    Parameters
    ----------
    imgs_np : numpy.array
        Numpy array of images
    pretrained_model : str, optional
        Name of the pretrained model to be used, by default "resnet50"
        ['resnet50', 'inception_v3', 'vgg19']
    pooling_method : str, optional
        Defines the last pooling layer that should be applied, by default 'avg'
        ['avg', 'max']
    
    Returns
    -------
    numpy.array
        Array of embeddings vectors. Each row represents embeddings for single input image
    """
    print('Input images shape: ', imgs_np.shape)
    pretrained_model = pretrained_model.lower()
    assert pretrained_model in ['inception_v3', 'vgg19']
    assert pooling_method in ['avg', 'max']

    model_args={
        'weights': 'imagenet',
        'include_top': False,
        'input_shape': imgs_np[0].shape
        }

    if pretrained_model=="inception_v3":
        base = InceptionV3(**model_args)
        from tensorflow.keras.applications.inception_v3 import preprocess_input
    elif pretrained_model=="vgg19":
        base = VGG19(**model_args)
        from tensorflow.keras.applications.vgg19 import preprocess_input

    feat_extractor = create_feat_extractor(base, pooling_method=pooling_method)

    imgs_np = preprocess_input(imgs_np)
    embeddings_np = feat_extractor.predict(imgs_np)
    print('Features shape: ', embeddings_np.shape)
    
    return embeddings_np

In [ ]:
# Feature extract는 GPU가 아닐경우 오래 걸림. CPU일 경우 imgs_np의 수를 조절
embeddings = extract_features(imgs_np, pretrained_model="inception_v3")

In [ ]:
embeddings.shape  # inception은 vector size가 2048, vgg19는 512

In [ ]:
embeddings[0:5]

# 쿼리 수행

In [ ]:
d = embeddings.shape[1]  # 차원
nb = embeddings.shape[0]  # 데이터 수
nq = 1  # 수행할 쿼리


In [ ]:
index = faiss.IndexFlatL2(d)   # 인덱스 생성
print('index.is_trained : ', index.is_trained)

In [ ]:
%time index.add(embeddings)  # Vector를 인덱스에 추가
print('index.ntotal : ', index.ntotal)

In [ ]:
file_order = 600  # 이 위치의 데이터로 쿼리 수행
xq = embeddings[file_order:file_order + 1] 

In [ ]:
print('image file : ' + filenames[file_order])

p_img = Image.open(filenames[file_order])
img =  np.array(p_img) 
plt.imshow(img)

In [ ]:
# 5개의 최근접 결과 출력
k = 5 

In [ ]:
# search 실행
D, I = index.search(xq, k)

In [ ]:
print(D, I)

In [ ]:
# 결과 정보 출력
def draw_image5(img_indexes, img_distance):
    print(img_indexes)
    w=10
    h=10
    fig=plt.figure(figsize=(12, 12))
    columns = 5
    rows = 1
    for i in range(1, columns*rows +1):
        print('image index : ' + str(img_indexes[0][i-1]))
        print('image distance : ' + str(img_distance[0][i-1]))
        
        print('image file : ' + filenames[img_indexes[0][i-1]])
        p_img = Image.open(filenames[img_indexes[0][i-1]])
        img =  np.array(p_img) 
        fig.add_subplot(rows, columns, i)
        plt.imshow(img)
    plt.show()
    
draw_image5(I, D)

# 외부 파일 업로드 - vector로 변환 처리
- 외부 파일을 업로드 또는 가져오기
- get_images로 numpy로 변환
- extract_features로 feature 추출
- search 수행하고 결과 출력

In [ ]:
# 미리 올려둔 파일 다운로드
urllib.request.urlretrieve(
  '<CHANGE_HERE>',
   "apple.jpg")
q_img = 'apple.jpg'

# 이미지를 numpy로 변환
q_img_np = get_images([q_img])

# Feature 추출
xq = extract_features(q_img_np, pretrained_model="inception_v3")
print(xq.shape)

# search 수행
D, I = index.search(xq, k)
print(D, I)

# query 이미지
img = Image.open(q_img)
size = (200, 200)
resized_img = img.resize(size)
display(resized_img)

# 결과 출력
draw_image5(I, D)
